In [1]:
import tensorflow as tf
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, LayerNormalization, Dense, Attention, MultiHeadAttention, Lambda
from tensorflow.keras.models import Model, Sequential
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import backend as K

2024-07-24 17:30:37.333523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 17:30:37.352294: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 17:30:37.356828: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 17:30:37.368086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 17:30:38.192427: W tensorflow/compiler/tf2

In [2]:
class TimeSeriesDataset:
    def __init__(self, root_dir, feature_names=[]):
        self.data = self.load_data(root_dir, feature_names)

    def load_data(self, root_dir, feature_names):
        data = []

        for individual_dir in sorted(os.listdir(root_dir)):
            individual_path = os.path.join(root_dir, individual_dir)
            for class_dir in sorted(os.listdir(individual_path)):
                class_path = os.path.join(individual_path, class_dir)
                if os.path.isdir(class_path):
                    for file in glob.glob(os.path.join(class_path, "*.csv")):
                        df = pd.read_csv(file, usecols=feature_names)
                        class_name = os.path.splitext(os.path.basename(file))[0]
                        df["class"] = class_name
                        data.append(df)


        # Concatenate all data frames into a single data frame
        data = pd.concat(data, ignore_index=True)
        
        return data

In [3]:
root_dir = "glove_data"
feature_names = [
    "flex_1", "flex_2", "flex_3", "flex_4", "flex_5",
    "GYRx", "GYRy", "GYRz",
    "ACCx", "ACCy", "ACCz"
]

dataset = TimeSeriesDataset(root_dir, feature_names).data
dataset = dataset.sort_values(by=["class"])

# filter_classes = ["deaf", "fine", "good", "goodbye", "hello"]
# dataset = dataset[dataset["class"].isin(filter_classes)]

x_train, y_train = dataset.iloc[:, :-1].values, dataset.iloc[:, -1].values

# x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))


timesteps = 150
n_features = 11
num_classes = len(np.unique(y_train))

x_train = x_train.reshape((-1, timesteps, n_features))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [4]:
def positional_encoding(length, depth):
    depth = int(depth)
    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
    
    angle_rates = 1 / (10000**depths)                # (1, depth)
    angle_rads = positions * angle_rates             # (pos, depth)
    
    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class AddPositionalEncoding(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def build(self, input_shape):
        _, seq_len, d_model = input_shape
        self.pos_encoding = positional_encoding(seq_len, d_model)
    
    def call(self, inputs):
        # Ensure positional encoding has the same shape as the input
        return inputs + self.pos_encoding[:tf.shape(inputs)[1], :tf.shape(inputs)[2]]

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super().get_config()
        return config

def create_model(timesteps, n_features, num_classes):
    inputs = Input(shape=(timesteps, n_features))
    
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = LSTM(units=128, return_sequences=True)(x)
    x = LSTM(units=128, return_sequences=True)(x)
    
    x = AddPositionalEncoding()(x)
    
    # MultiHeadAttention layer
    attn_output = MultiHeadAttention(num_heads=8, key_dim=128)(x, x, x)
    x = LayerNormalization()(attn_output + x)
    
    x = Dense(units=128, activation='relu')(x)
    
    # Global Attention layer
    attn = Attention()([x, x])
    x = LayerNormalization()(attn + x)
    
    # Global average pooling to reduce sequence dimension
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [5]:
model = create_model(timesteps, n_features, num_classes)
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

I0000 00:00:1721817046.125578   13575 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721817046.171009   13575 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721817046.171197   13575 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721817046.172434   13575 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 11)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 148, 64)   │      2,176 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 74, 64)    │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 74, 128)   │     98,816 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 74, 128)   │    131,584 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_positional_enc… │ (None, 74, 128)   │          0 │ lstm_1[0][0]      │
│ (AddPositionalEnco… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 74, 128)   │    527,488 │ add_positional_e… │
│ (MultiHeadAttentio… │                   │            │ add_positional_e… │
│                     │                   │            │ add_positional_e… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 128)   │          0 │ multi_head_atten… │
│                     │                   │            │ add_positional_e… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 74, 128)   │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 74, 128)   │     16,512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 74, 128)   │          0 │ dense[0][0],      │
│ (Attention)         │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 74, 128)   │          0 │ attention[0][0],  │
│                     │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 74, 128)   │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 40)        │      5,160 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 782,248 (2.98 MB)

 Trainable params: 782,248 (2.98 MB)

...

Epoch 1/10


2024-07-24 17:30:49.634328: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1721817049.787572   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.831044   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.832062   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.833059   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.834111   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.835107   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.836109   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817049.837126   13648 gpu_t

  5/225 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.5433 - loss: 1.9561    

W0000 00:00:1721817051.019854   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.028647   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.029569   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.036176   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.037091   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.038450   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.044776   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.047340   13648 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1721817051.049413   13648 gp

225/225 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9735 - loss: 0.1211 - val_accuracy: 1.0000 - val_loss: 1.0493e-04
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 1.0000 - loss: 8.7903e-05 - val_accuracy: 1.0000 - val_loss: 5.4306e-05
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 1.0000 - loss: 4.8529e-05 - val_accuracy: 1.0000 - val_loss: 3.4932e-05
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 1.0000 - loss: 3.2012e-05 - val_accuracy: 1.0000 - val_loss: 2.4584e-05
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 1.0000 - loss: 2.2855e-05 - val_accuracy: 1.0000 - val_loss: 1.8223e-05
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 1.0000 - loss: 1.7046e-05 - val_accuracy: 1.0000 - val_loss: 1.4108e-05
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 1.0000 - loss: 1.3243e-05 - val_accuracy: 1.0000 - val_loss: 1.1039e-05
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - 